<a href="https://colab.research.google.com/github/tuenv193176/testfom186/blob/main/datn_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/googledrive')

Mounted at /content/googledrive


In [ ]:
pip install --upgrade deap

In [ ]:
!pip install -q lazypredict

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from deap import base, creator, tools, algorithms
from sklearn.metrics import mean_squared_error
from lazypredict.Supervised import LazyRegressor
import random
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Đọc dữ liệu từ file CSV
file_path = '/content/googledrive/MyDrive/chot1.csv'
data = pd.read_csv(file_path)
data

,Freq [GHz],"dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='1mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='4.66666666666667mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='8.33333333333333mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='12mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='1mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='4.66666666666667mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='8.33333333333333mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='12mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='8.33333333333333mm' l12='1mm' l21='1mm' l22='1mm'",...,"dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='4.66666666666667mm' l12='12mm' l21='8.33333333333333mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='4.66666666666667mm' l12='12mm' l21='12mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='1mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='4.66666666666667mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='8.33333333333333mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='12mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='1mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='4.66666666666667mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='8.33333333333333mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='12mm' l22='12mm'"
0,2.00,-0.21,-0.22,-0.24,-0.28,-0.24,-0.24,-0.26,-0.29,-0.28,...,-0.56,-0.58,-0.63,-0.63,-0.64,-0.65,-0.77,-0.74,-0.73,-0.74
1,2.01,-0.21,-0.22,-0.25,-0.28,-0.24,-0.25,-0.26,-0.30,-0.29,...,-0.58,-0.59,-0.65,-0.65,-0.65,-0.67,-0.79,-0.76,-0.76,-0.77
2,2.02,-0.22,-0.23,-0.25,-0.29,-0.24,-0.25,-0.27,-0.30,-0.29,...,-0.59,-0.61,-0.67,-0.66,-0.67,-0.68,-0.82,-0.78,-0.78,-0.79
3,2.03,-0.22,-0.23,-0.25,-0.29,-0.24,-0.25,-0.27,-0.31,-0.29,...,-0.61,-0.63,-0.69,-0.68,-0.69,-0.70,-0.85,-0.81,-0.80,-0.82
4,2.04,-0.22,-0.23,-0.25,-0.29,-0.25,-0.25,-0.27,-0.31,-0.30,...,-0.63,-0.64,-0.72,-0.70,-0.71,-0.73,-0.88,-0.83,-0.83,-0.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,5.96,-2.05,-2.07,-1.27,-2.12,-1.65,-1.88,-1.58,-1.55,-1.48,...,-0.99,-0.76,-0.81,-1.39,-0.77,-0.76,-0.82,-2.73,-0.79,-0.76
397,5.97,-2.04,-2.06,-1.27,-2.12,-1.66,-1.88,-1.57,-1.55,-1.49,...,-0.97,-0.77,-0.81,-1.42,-0.77,-0.76,-0.82,-2.83,-0.79,-0.76
398,5.98,-2.03,-2.06,-1.26,-2.11,-1.66,-1.87,-1.56,-1.54,-1.49,...,-0.94,-0.77,-0.82,-1.44,-0.77,-0.76,-0.82,-2.93,-0.79,-0.76
399,5.99,-2.02,-2.06,-1.26,-2.11,-1.66,-1.86,-1.55,-1.54,-1.50,...,-0.92,-0.78,-0.82,-1.47,-0.78,-0.77,-0.82,-3.05,-0.79,-0.76


In [ ]:
filtered_data = data[(data['Freq [GHz]'] >= 2.4) & (data['Freq [GHz]'] <= 3.0) |
                     (data['Freq [GHz]'] >= 5.1) & (data['Freq [GHz]'] <= 5.6)]

In [ ]:
filtered_data = filtered_data.drop(columns=['Freq [GHz]'])
filtered_data

,"dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='1mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='4.66666666666667mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='8.33333333333333mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='12mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='1mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='4.66666666666667mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='8.33333333333333mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='12mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='8.33333333333333mm' l12='1mm' l21='1mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='8.33333333333333mm' l12='1mm' l21='4.66666666666667mm' l22='1mm'",...,"dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='4.66666666666667mm' l12='12mm' l21='8.33333333333333mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='4.66666666666667mm' l12='12mm' l21='12mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='1mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='4.66666666666667mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='8.33333333333333mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='12mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='1mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='4.66666666666667mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='8.33333333333333mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='12mm' l22='12mm'"
40,-0.29,-0.32,-0.39,-0.52,-0.37,-0.39,-0.45,-0.59,-0.54,-0.45,...,-2.14,-2.30,-2.76,-2.65,-2.70,-2.74,-4.01,-3.69,-3.65,-3.68
41,-0.29,-0.32,-0.40,-0.54,-0.38,-0.39,-0.46,-0.60,-0.55,-0.46,...,-2.23,-2.41,-2.89,-2.77,-2.83,-2.87,-4.23,-3.89,-3.84,-3.87
42,-0.29,-0.32,-0.40,-0.55,-0.38,-0.40,-0.46,-0.61,-0.56,-0.47,...,-2.33,-2.52,-3.03,-2.90,-2.97,-3.00,-4.46,-4.09,-4.04,-4.07
43,-0.30,-0.33,-0.41,-0.56,-0.39,-0.41,-0.47,-0.63,-0.57,-0.48,...,-2.43,-2.63,-3.18,-3.04,-3.11,-3.14,-4.71,-4.31,-4.26,-4.29
44,-0.30,-0.33,-0.42,-0.57,-0.39,-0.41,-0.48,-0.64,-0.58,-0.49,...,-2.54,-2.75,-3.33,-3.19,-3.26,-3.29,-4.96,-4.54,-4.49,-4.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,-2.51,-2.23,-1.44,-2.01,-1.67,-2.04,-1.76,-1.62,-1.29,-1.42,...,-1.59,-0.69,-0.75,-0.92,-0.79,-0.73,-0.78,-1.08,-0.84,-0.82
357,-2.50,-2.22,-1.43,-2.02,-1.67,-2.04,-1.76,-1.62,-1.29,-1.43,...,-1.60,-0.68,-0.75,-0.92,-0.78,-0.73,-0.78,-1.10,-0.84,-0.82
358,-2.49,-2.21,-1.42,-2.02,-1.67,-2.04,-1.77,-1.62,-1.29,-1.43,...,-1.60,-0.68,-0.75,-0.93,-0.78,-0.73,-0.78,-1.11,-0.83,-0.82
359,-2.47,-2.21,-1.42,-2.03,-1.67,-2.04,-1.77,-1.62,-1.30,-1.43,...,-1.61,-0.68,-0.75,-0.93,-0.78,-0.73,-0.78,-1.13,-0.83,-0.81


In [ ]:
absolute_sum_s11 = filtered_data.abs().sum()
absolute_sum_s11

dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='1mm' l22='1mm'                     174.28
dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='4.66666666666667mm' l22='1mm'      157.12
dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='8.33333333333333mm' l22='1mm'      129.45
dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='12mm' l22='1mm'                    172.30
dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='1mm' l22='1mm'      133.17
                                                                                                 ...  
dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='12mm' l22='12mm'   640.95
dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='1mm' l22='12mm'                  626.34
dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='4.66666666666667mm' l22='12mm'   651.28
dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='8.33333333

In [ ]:
def extract_params(column_name):
    pattern = r"h1='([\d.]+)mm' h2='([\d.]+)mm' l11='([\d.]+)mm' l12='([\d.]+)mm' l21='([\d.]+)mm' l22='([\d.]+)mm'"
    match = re.search(pattern, column_name)
    if match:
        return [float(match.group(i)) for i in range(1, 7)]
    return [None] * 6

In [ ]:
data_new = pd.DataFrame([extract_params(col) for col in absolute_sum_s11.index],
                        columns=['h1', 'h2', 'l11', 'l12', 'l21', 'l22'])

data_new['sum_s11'] = absolute_sum_s11.values
data_new

,h1,h2,l11,l12,l21,l22,sum_s11
0,5.00,0.50,1.00,1.00,1.00,1.00,174.28
1,5.00,0.50,1.00,1.00,4.67,1.00,157.12
2,5.00,0.50,1.00,1.00,8.33,1.00,129.45
3,5.00,0.50,1.00,1.00,12.00,1.00,172.30
4,5.00,0.50,4.67,1.00,1.00,1.00,133.17
...,...,...,...,...,...,...,...
2299,8.00,1.50,8.33,12.00,12.00,12.00,640.95
2300,8.00,1.50,12.00,12.00,1.00,12.00,626.34
2301,8.00,1.50,12.00,12.00,4.67,12.00,651.28
2302,8.00,1.50,12.00,12.00,8.33,12.00,653.68


In [ ]:
X = data_new[['h1', 'h2', 'l11', 'l12', 'l21', 'l22']]
y = data_new['sum_s11']
X

,h1,h2,l11,l12,l21,l22
0,5.00,0.50,1.00,1.00,1.00,1.00
1,5.00,0.50,1.00,1.00,4.67,1.00
2,5.00,0.50,1.00,1.00,8.33,1.00
3,5.00,0.50,1.00,1.00,12.00,1.00
4,5.00,0.50,4.67,1.00,1.00,1.00
...,...,...,...,...,...,...
2299,8.00,1.50,8.33,12.00,12.00,12.00
2300,8.00,1.50,12.00,12.00,1.00,12.00
2301,8.00,1.50,12.00,12.00,4.67,12.00
2302,8.00,1.50,12.00,12.00,8.33,12.00


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Chuẩn hóa dữ liệu đầu ra
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

In [ ]:
lazy_model = LazyRegressor(ignore_warnings=True, custom_metric=None, verbose=0, predictions=True, regressors='all')
models, predictions = lazy_model.fit(X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled)
models

100%|██████████| 42/42 [00:06<00:00,  6.09it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000268 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28
[LightGBM] [Info] Number of data points in the train set: 1843, number of used features: 6
[LightGBM] [Info] Start training from score 0.000000


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
HistGradientBoostingRegressor,0.97,0.97,0.18,0.33
LGBMRegressor,0.97,0.97,0.18,0.10
XGBRegressor,0.97,0.97,0.18,0.15
RandomForestRegressor,0.97,0.97,0.19,0.43
BaggingRegressor,0.97,0.97,0.20,0.09
ExtraTreesRegressor,0.96,0.97,0.20,0.58
DecisionTreeRegressor,0.96,0.96,0.21,0.06
MLPRegressor,0.95,0.95,0.23,1.89
NuSVR,0.95,0.95,0.23,0.31


In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(1))

In [ ]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=MeanSquaredError())

In [ ]:
history = model.fit(X_train_scaled, y_train_scaled, epochs=1000, batch_size=16, validation_split=0.2, verbose=0)

In [ ]:
loss = model.evaluate(X_test_scaled, y_test_scaled)
print(f'Mean Squared Error on Test Set: {loss}')

15/15 [==============================] - 0s 2ms/step - loss: 0.0383
Mean Squared Error on Test Set: 0.03833857923746109


In [ ]:
y_test_pred_scaled = model.predict(X_test_scaled)
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).flatten()

# Dự đoán trên toàn bộ dữ liệu đã được chuẩn hóa
X_scaled = scaler_X.transform(X)
y_pred_scaled = model.predict(X_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Đảm bảo các giá trị đầu ra được so sánh đúng
comparison = pd.DataFrame({'Actual': y, 'Predicted': y_pred})
print(comparison.head())

72/72 [==============================] - 0s 4ms/step
   Actual  Predicted
0  174.28     172.25
1  157.12     158.95
2  129.45     130.59
3  172.30     174.16
4  133.17     131.37


In [ ]:
first_sample = data_new.iloc[509]
X_first_sample = first_sample[['h1', 'h2', 'l11', 'l12', 'l21', 'l22']].values.reshape(1, -1)
y_first_sample_actual = first_sample['sum_s11']

# Chuẩn hóa mẫu đầu vào
X_first_sample_scaled = scaler_X.transform(X_first_sample)

# Dự đoán giá trị sum_s11 cho mẫu đầu tiên
y_first_sample_pred_scaled = model.predict(X_first_sample_scaled)
y_first_sample_pred = scaler_y.inverse_transform(y_first_sample_pred_scaled)

print(f"Actual sum_s11: {y_first_sample_actual}")
print(f"Predicted sum_s11: {y_first_sample_pred[0][0]}")

1/1 [==============================] - 0s 32ms/step
Actual sum_s11: 1089.3506641285803
Predicted sum_s11: 922.674560546875


In [ ]:
def predict_sum_s11(h1, h2, l11, l12, l21, l22):
    input_sample = pd.DataFrame([[h1, h2, l11, l12, l21, l22]], columns=['h1', 'h2', 'l11', 'l12', 'l21', 'l22'])
    input_scaled = scaler_X.transform(input_sample)
    predicted_sum_s11_scaled = model.predict(input_scaled)
    predicted_sum_s11 = scaler_y.inverse_transform(predicted_sum_s11_scaled)
    return predicted_sum_s11[0][0]

In [ ]:
input_values = [5, 0.5, 1, 1, 1, 12]
predicted_sum_s11 = predict_sum_s11(*input_values)
print(f'Predicted sum_s11 for input {input_values}: {predicted_sum_s11}')

1/1 [==============================] - 0s 25ms/step
Predicted sum_s11 for input [5, 0.5, 1, 1, 1, 12]: 181.12330627441406


In [ ]:
# prompt: applying the GA algorithm to optimize the inputs h1, h2, l11, l12, l21, l22 to help the ANN model accurately predict the largest sum_s11 value. All variables are positive. h2 is the smallest value.

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, 12)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 6)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    h1, h2, l11, l12, l21, l22 = individual
    predicted_sum_s11 = predict_sum_s11(h1, h2, l11, l12, l21, l22)
    return predicted_sum_s11,

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=100)
NGEN = 100
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(population + offspring, k=len(population))

best_individual = tools.selBest(population, k=1)[0]
best_h1, best_h2, best_l11, best_l12, best_l21, best_l22 = best_individual
best_sum_s11 = predict_sum_s11(best_h1, best_h2, best_l11, best_l12, best_l21, best_l22)

print(f"Best h1: {best_h1}")
print(f"Best h2: {best_h2}")
print(f"Best l11: {best_l11}")
print(f"Best l12: {best_l12}")
print(f"Best l21: {best_l21}")
print(f"Best l22: {best_l22}")
print(f"Best sum_s11: {best_sum_s11}")


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
1/1 [==============================] - 0s 27ms/step
Best h1: -21.339057924964212
Best h2: 20.868863808117354
Best l11: -7.237708725696622
Best l12: -8.414373747354134
Best l21: 3.4738578358882046
Best l22: -4.7503493336686144
Best sum_s11: -1795.1234130859375
